# Обучение моделей NLP

## Категоризация текстовых документов

### Импорт библиотек

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

### Обучающие данные

In [ ]:
# Пример данных для обучения (новостные статьи и соответствующие им категории)
news_articles = [
    ("В преддверии предстоящих выборов политические партии активизировали свои кампании. Кандидаты проводят множество митингов и дебатов, чтобы привлечь внимание избирателей. Электорат активно обсуждает предвыборные программы и кандидатов в социальных сетях. Ожидается, что эти выборы будут иметь значительное влияние на политическую обстановку в стране. Аналитики предсказывают жесткую борьбу между основными политическими силами за голоса избирателей.", "Политика"),
    ("В свете последних событий многие страны сталкиваются с необходимостью провести серьезные политические реформы. Политические лидеры рассматривают возможные пути реализации этих изменений. Обсуждается внедрение новых законов и политических систем, которые могут стимулировать экономический рост и улучшить качество жизни граждан. Однако возникают и опасения относительно стабильности и устойчивости таких изменений.", "Политика"),
    ("Современная мировая политическая сцена характеризуется многообразием вызовов и возможностей. Страны сталкиваются с такими проблемами, как глобальное потепление, международный терроризм и экономические кризисы. Одновременно возникают новые возможности для сотрудничества и развития международных отношений. Стремление к миру и стабильности остается приоритетом для многих государств.", "Политика"),
    ("С развитием технологий цифровой революции политические лидеры вынуждены адаптировать свои стратегии. Использование социальных медиа и интернета становится неотъемлемой частью избирательной кампании. Появляются новые методы аналитики и взаимодействия с избирателями. Однако с ростом цифровой активности возникают и новые угрозы, такие как цифровые атаки и фейковые новости.", "Политика"),
    ("Молодежь играет все более важную роль в политической жизни многих стран. Молодые активисты активно выступают за свои права и интересы. Они используют социальные сети для мобилизации и организации политических движений. Однако молодежные инициативы часто сталкиваются с препятствиями со стороны старшего поколения политических деятелей и установленных политических систем.", "Политика"),
    ("В истории науки было множество важных открытий, которые радикально изменили наше понимание мира. Открытия, такие как законы Ньютона о движении и гравитации, теория относительности Эйнштейна, открытие ДНК и структуры атома, перевернули представления об устройстве Вселенной и жизни. Эти ключевые моменты в истории науки оставили неизгладимый след и продолжают вдохновлять ученых на новые открытия и исследования.", "Наука"),
    ("Современная наука находится на стыке различных дисциплин и технологий, что создает уникальные возможности для инноваций и открытий. Тенденции в научных исследованиях отражают интерес к таким областям, как искусственный интеллект, биотехнологии, космология, квантовая физика и экология. Эти направления открывают новые горизонты для понимания мира и развития человечества.", "Наука"),
    ("Научные открытия играют ключевую роль в развитии медицины и улучшении здоровья людей. Открытия в области генетики, нейронауки, фармакологии и медицинской технологии позволяют разрабатывать новые методы лечения, предупреждать заболевания и повышать качество медицинской помощи. Научные достижения в области здоровья продолжают способствовать увеличению продолжительности жизни и повышению качества жизни людей по всему миру.", "Наука"),
    ("Современные экологические исследования сталкиваются с серьезными вызовами, связанными с изменением климата, потерей биоразнообразия и загрязнением окружающей среды. Ученые изучают воздействие человеческой деятельности на экосистемы и разрабатывают стратегии для их сохранения и восстановления. Экологические исследования играют ключевую роль в формировании политики охраны окружающей среды и принятии мер для предотвращения экологических катастроф.", "Наука"),
    ("Наука и технологии взаимосвязаны и взаимообогащают друг друга, способствуя технологическому прогрессу. Исследования в области физики, химии, математики и информационных технологий положили основу для развития современных технологий, таких как компьютеры, мобильные устройства, медицинские приборы и возобновляемые источники энергии. В свою очередь, технологии открывают новые возможности для научных исследований и помогают ученым решать сложные проблемы.", "Наука"),
    ("Здоровый образ жизни играет решающую роль в обеспечении долголетия и общего благополучия. Регулярные физические упражнения, сбалансированное питание, достаточный отдых и отказ от вредных привычек, таких как курение и чрезмерное употребление алкоголя, способствуют поддержанию здоровья сердечно-сосудистой системы, иммунитета и психического благополучия. Ведение здорового образа жизни - это инвестиция в будущее, которая позволяет предотвращать многие хронические заболевания и наслаждаться полноценной жизнью.", "Здоровье"),
    ("Ментальное здоровье играет ключевую роль в общем благополучии человека. Стресс, депрессия, тревожность и другие психические проблемы могут оказать негативное влияние на качество жизни и способность функционировать в обществе. Забота о ментальном здоровье включает в себя умение управлять стрессом, находить сбалансированные решения и обращаться за помощью к специалистам при необходимости. Поддержание здоровья как физического, так и психического, является ключевым аспектом общего благополучия.", "Здоровье"),
    ("Работа занимает значительную часть времени в жизни человека, поэтому забота о здоровье на рабочем месте играет важную роль в общем благополучии. Эргономичные условия труда, регулярные перерывы и физические упражнения на рабочем месте помогают предотвращать стресс, усталость и другие проблемы, связанные с долгосрочной работой за компьютером или в офисе. Кроме того, создание поддерживающей атмосферы и культуры заботы о здоровье на рабочем месте способствует повышению производительности и улучшению качества работы.", "Здоровье"),
    ("Правильное питание играет важную роль в поддержании здоровья и предотвращении заболеваний. Балансированное потребление питательных веществ, включая фрукты, овощи, белки, углеводы и жиры, помогает поддерживать оптимальный вес, укреплять иммунитет и уменьшать риск развития сердечно-сосудистых заболеваний, диабета и других хронических заболеваний. Правильное питание - это не просто диета, это образ жизни, который способствует долголетию и общему благополучию.", "Здоровье"),
    ("Здоровье семьи играет важную роль в общем благополучии каждого члена семьи. Создание поддерживающей среды, включающей правильное питание, регулярные физические упражнения и заботу о ментальном благополучии, способствует укреплению взаимоотношений и предотвращению различных заболеваний. Забота о здоровье семьи начинается с осознания важности здорового образа жизни и принятия соответствующих мер для его поддержания.", "Здоровье"),
    ("В последнее время мир спорта был свидетелем впечатляющих достижений и удивительных рекордов. Спортсмены из различных дисциплин продемонстрировали невероятную выносливость и мастерство. Новые рекорды были установлены в плавании, легкой атлетике, футболе и других видах спорта. Эти достижения не только вдохновляют новое поколение спортсменов, но и заставляют весь мир удивляться и восхищаться.", "Спорт"),
    ("Период трансферов в мире спорта всегда волнует фанатов. В этом году многие известные спортсмены объявили о своих переходах к новым командам и клубам. Футбольные звезды, баскетбольные чемпионы и теннисные гении меняют свои альянсы в поисках новых вызовов и возможностей. Эти переходы станут основой для увлекательных соревнований и эмоциональных моментов в предстоящем сезоне.", "Спорт"),
    ("Спорт играет важную роль в поддержании здорового образа жизни. Регулярные физические упражнения помогают укрепить мышцы, улучшить сердечно-сосудистую систему и поддерживать оптимальный вес. Однако спорт также оказывает положительное воздействие на психическое здоровье, помогая справиться со стрессом, улучшить настроение и повысить самооценку. Поэтому ведение активного образа жизни, включая занятия спортом, является ключом к полноценной и здоровой жизни.", "Спорт"),
    ("В мире спорта есть множество легендарных фигур, чьи достижения останутся в памяти поколений. Эти спортсмены преодолели невероятные препятствия, чтобы достичь вершин своих дисциплин. Их истории вдохновляют миллионы людей по всему миру на поиск своего собственного успеха и самосовершенствования. Именно благодаря таким героям спорт становится не только состязанием, но и источником вдохновения.", "Спорт"),
    ("Современные технологии играют все более важную роль в мире спорта. От электронных устройств для тренировок до видеотехнологий для судейства, инновации проникают во все аспекты спортивной деятельности. Эти технологии не только улучшают производительность спортсменов, но и изменяют правила игры и способы взаимодействия с болельщиками. В будущем можно ожидать еще больших изменений в мире спорта благодаря постоянному развитию технологий.", "Спорт"),
]

### Обработка данных

In [ ]:
# Предварительная обработка текста
def preprocess_text(text):
    stop_words = set(stopwords.words('russian'))
    tokens = nltk.word_tokenize(text)
    tokens = [token.lower() for token in tokens if token.isalpha()]
    tokens = [token for token in tokens if token not in stop_words]
    stemmer = SnowballStemmer("russian")
    stemmed_tokens = [stemmer.stem(token) for token in tokens]
    return ' '.join(stemmed_tokens)

# Подготовка данных для обучения
X = [preprocess_text(article[0]) for article in news_articles]
y = [article[1] for article in news_articles]

### Обучение данных

In [ ]:
# Преобразование текстов в TF-IDF признаки
tfidf_vectorizer = TfidfVectorizer()
X_tfidf = tfidf_vectorizer.fit_transform(X)

# Обучение модели
classifier = MultinomialNB()
classifier.fit(X_tfidf, y)

### Использование модели

- Здоровье: "Ученые обнаружили, что регулярные утренние пробежки могут снизить риск развития сердечно-сосудистых заболеваний на 30%."

- Наука: "Новое исследование показывает, что использование искусственного интеллекта позволяет увеличить эффективность процессов в области медицины на 50%."

- Политика: "В результате последних выборов парламент получил сильное большинство, что может привести к внесению кардинальных изменений в налоговую политику страны."

- Спорт: "Звездный футболист подписал новый контракт с клубом, став самым высокооплачиваемым игроком в истории футбола."

In [ ]:
new_article_test = [
    "Ученые обнаружили, что регулярные утренние пробежки могут снизить риск развития сердечно-сосудистых заболеваний на 30%",
    "Новое исследование показывает, что использование искусственного интеллекта позволяет увеличить эффективность процессов в области медицины на 50%.",
    "В результате последних выборов парламент получил сильное большинство, что может привести к внесению кардинальных изменений в налоговую политику страны.",
    "Звездный футболист подписал новый контракт с клубом, став самым высокооплачиваемым игроком в истории футбола.",
]

In [ ]:
new_article = preprocess_text(new_article_test[3])
new_article_vectorized = tfidf_vectorizer.transform([new_article])
prediction = classifier.predict(new_article_vectorized)
print("Прогноз категории для новой статьи:", prediction[0])

## Cистема "Чат-бот"

### Импорт библиотек

In [ ]:
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline

### Обучающие данные

In [ ]:
questions = [
    "Как тебя зовут?",
    "Как дела?",
    "Что ты умеешь?",
    "Чем занимаешься?"
]

answers = [
    "Меня зовут ChatGPT. Я искусственный интеллект, созданный OpenAI.",
    "У меня все отлично, спасибо за спрос.",
    "Я могу отвечать на вопросы, помогать в решении задач и многое другое.",
    "Я разговариваю с вами! :)"
]

### Обучение модели

In [ ]:
# Векторизация текста и обучение модели
vectorizer = TfidfVectorizer()
classifier = MultinomialNB()

# Создание пайплайна
model = make_pipeline(vectorizer, classifier)

# Обучение модели
model.fit(questions, answers)

### Использование модели

In [ ]:
# Функция для предсказания ответа на вопрос
def get_response(question):
    return model.predict([question])[0]

# Тестирование модели
while True:
    user_input = input("Вы: ")
    if user_input.lower() == 'выход':
        break
    response = get_response(user_input)
    print("Бот:", response)

## Ответы на вопросы по тексту

In [ ]:
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline

### Обучающие данные

In [ ]:
training_text = """
Python - это высокоуровневый язык программирования общего назначения, 
который широко используется для разработки веб-приложений, научных 
вычислений, искусственного интеллекта и других сфер. Он имеет простой 
и понятный синтаксис, что делает его очень популярным среди начинающих 
и опытных программистов.

NLP (Natural Language Processing) - это область искусственного интеллекта, 
которая занимается взаимодействием между компьютерами и естественными 
языками человека. Она включает в себя обработку текста, анализ смысла, 
разговорное распознавание речи и многое другое.
"""

### Обработка текста

In [ ]:
# Функция для предварительной обработки текста
def preprocess_text(text):
    # Разделение текста на предложения
    sentences = nltk.sent_tokenize(text)
    # Преобразование предложений в нижний регистр
    sentences = [sentence.lower() for sentence in sentences]
    return sentences

# Получение предобработанных предложений
sentences = preprocess_text(training_text)

### Обучение модели

In [ ]:
# Векторизация текста и обучение модели
vectorizer = TfidfVectorizer()
classifier = MultinomialNB()

# Создание пайплайна
model = make_pipeline(vectorizer, classifier)

# Обучение модели
model.fit(sentences, list(range(len(sentences))))

### Использование модели

In [ ]:
# Функция для предсказания ответа на вопрос
def get_response(question):
    # Предобработка вопроса
    question = preprocess_text(question)
    # Предсказание ответа на основе вопроса
    predicted_sentence_index = model.predict(question)[0]
    return sentences[predicted_sentence_index]

# Тестирование модели
while True:
    user_input = input("Вы: ")
    if user_input.lower() == 'выход':
        break
    response = get_response(user_input)
    print("Бот:", response)